### Pacman

In [ ]:
import time
import heapq
from typing import Tuple, List

### IDS implementation

In [ ]:
def make_move(board_state, agent, move, cols):
    idxp = board_state.index(agent)
    target = None
    
    if move == 'L':
        target = idxp - 1
    elif move == 'D':
        target = idxp + cols
    elif move == 'R':
        target = idxp + 1
    elif move == 'U':
        target = idxp - cols
        
    if (
        target is not None
        and target >= 0
        and target < len(board_state)
        and board_state[target] in ('.', '3', f"{'1' if agent == 'P' else '2'}")
    ):
        new_board_state = list(board_state)
        new_board_state[target] = agent
        new_board_state[idxp] = '.'
        if f"{'1' if agent == 'P' else '2'}" not in new_board_state and '3' not in new_board_state:
            new_board_state[target] = '.'
        return ''.join(new_board_state)
    return False

def dfs(board_state,cols,agent,move,psteps,qsteps,visited,curr_depth,depth):

  if curr_depth == depth:
    return -1,-1

  # check if move is possible 
  if agent != "":
    board_state = make_move(board_state,agent,move,cols)
    if not board_state:
      return -1,-1

  if '1' not in board_state and '2' not in board_state and '3' not in board_state: 
    return psteps,qsteps
  
  # add to explored 
  if board_state in visited:
    if curr_depth < visited[board_state]:
      visited[board_state] = curr_depth 
    else:
      return -1,-1
  visited[board_state] = curr_depth

  if 'P' in board_state:
    for move in ['U','R','D','L']:
      result = dfs(board_state,cols,'P',move,psteps+move,qsteps,visited,curr_depth+1,depth)
      if result != (-1,-1):
        return result

  if 'Q' in board_state:  
    for move in ['U','R','D','L']:  
      result = dfs(board_state,cols,'Q',move,psteps,qsteps+move,visited,curr_depth+1,depth)
      if result != (-1,-1):
        return result

  return -1,-1
  

def IDS(board_map: List[str]) -> Tuple[str, str]:
    p_steps, q_steps = list(), list()
    # join board_map to make start_state
    start_state = []
    cols = len(board_map[0])-1
    for s in board_map:
        start_state.append(s[0:len(s)-1])
    start_state = ''.join(start_state) + '%'
    
    # search
    depth_max = 2
    while(True): 
        visited = {}
        psteps,qsteps = dfs(start_state,cols,"","","","",visited,0,depth_max)
        if psteps!=-1:
          return psteps,qsteps
        depth_max+=1
  


### A* implementation

In [ ]:
def get_successors(board_state,cols,psteps,qsteps):
    
    neighbors =[]
    
    if 'P' in board_state:
      idxp = board_state.index('P') 
      agent='P'

      # UP P - subtract 13 to go up AND only if '.' or '1' or '3'
      if(idxp-cols >= 0 and (board_state[idxp-cols]=='.' or board_state[idxp-cols]=='3' or board_state[idxp-cols]=='1')):
          l = list(board_state)
          l[idxp-cols]= agent
          l[idxp] = '.'
          #check if all 1 and 3 are over
          if not '1' in l and not '3' in l:
              l[idxp-cols] = '.'
          new = ''.join(l)
          neighbors.append((new,'P','U',psteps+'U',qsteps))  
          
      # RIGHT P - RIGHT add 1 to go right AND only if '.' or '1' or '3'
      if(idxp+1 < len(board_state) and (board_state[idxp+1]=='.' or board_state[idxp+1]=='3' or board_state[idxp+1]=='1')):
          l = list(board_state)
          l[idxp+1]= agent
          l[idxp] = '.'
          #check if all 1 and 3 are over
          if not '1' in l and not '3' in l:
              l[idxp+1] = '.'

          new = ''.join(l)
          neighbors.append((new,'P','R',psteps+'R',qsteps))  
          
      # DOWN P - DOWN add 13 to go right AND only if '.' or '1' or '3'
      if(idxp+cols < len(board_state) and (board_state[idxp+cols]=='.' or board_state[idxp+cols]=='3' or board_state[idxp+cols]=='1')):
          l = list(board_state)
          l[idxp+cols] = agent
          l[idxp] = '.'
          #check if all 1 and 3 are over
          if not '1' in l and not '3' in l:
              l[idxp+cols] = '.'

          new = ''.join(l)
          neighbors.append((new,'P','D',psteps+'D',qsteps))  
          
      # LEFT P - LEFT subtract 1 to go left AND only if '.' or '1' or '3'
      if(idxp-1 >= 0 and (board_state[idxp-1]=='.' or board_state[idxp-1]=='3' or board_state[idxp-1]=='1')):
          l = list(board_state)
          l[idxp-1] = agent
          l[idxp]= '.'
          #check if all 1 and 3 are over
          if not '1' in l and not '3' in l:
              l[idxp-1] = '.'

          new = ''.join(l)
          neighbors.append((new,'P','L',psteps+'L',qsteps))  
          
    if 'Q' in board_state:
        idxq = board_state.index('Q')
        agent = 'Q'

        # UP Q - up subtract 13 to go up AND only if '.' or '1' or '3'
        if(idxq-cols >= 0 and (board_state[idxq-cols]=='.' or board_state[idxq-cols]=='3' or board_state[idxq-cols]=='2')):
            l = list(board_state)
            l[idxq-cols]= agent
            l[idxq] = '.'
            # check if all 2 and 3 are over
            if not '2' in l and not '3' in l:
                l[idxq-cols] = '.'
      
            new = ''.join(l)
            neighbors.append((new,'Q','U',psteps,qsteps+'U'))  
          
        # RIGHT Q - RIGHT add 1 to go right AND only if '.' or '2' or '3'
        if(idxq+1 < len(board_state) and (board_state[idxq+1]=='.' or board_state[idxq+1]=='3' or board_state[idxq+1]=='2')):
            l = list(board_state)
            l[idxq+1]= agent
            l[idxq] = '.'
            # check if all 2 and 3 are over
            if not '2' in l and not '3' in l:
                l[idxq+1] = '.'
      
            new = ''.join(l)
            neighbors.append((new,'Q','R',psteps,qsteps+'R'))  
          
        # DOWN Q - DOWN add 13 to go right AND only if '.' or '2' or '3'
        if(idxq+cols < len(board_state) and (board_state[idxq+cols]=='.' or board_state[idxq+cols]=='3' or board_state[idxq+cols]=='2')):
            l = list(board_state)
            l[idxq+cols] = agent
            l[idxq] = '.'
            # check if all 2 and 3 are over
            if not '2' in l and not '3' in l:
                l[idxq+cols] = '.'
      
            new = ''.join(l)
            neighbors.append((new,'Q','D',psteps,qsteps+'D'))  
          
        # LEFT Q - LEFT subtract 1 to go left AND only if '.' or '2' or '3'
        if(idxq-1 >= 0 and (board_state[idxq-1]=='.' or board_state[idxq-1]=='3' or board_state[idxq-1]=='2')):
            l = list(board_state)
            l[idxq-1] = agent
            l[idxq]= '.'
            # check if all 2 and 3 are over
            if not '2' in l and not '3' in l:
                l[idxq-1] = '.'
      
            new = ''.join(l)
            neighbors.append((new,'Q','L',psteps,qsteps+'L'))  
  
    return neighbors



def heuristic(board_state,cols):

    p_distance = 0
    q_distance = 0

    if 'P' in board_state:
      index = board_state.index("P")
      row,col = divmod(index,cols)
      for char in board_state:
        if char in ['1','3']:
          num_index = board_state.index(char)
          num_row,num_col = divmod(num_index,cols)
          p_distance = min(p_distance,(abs(row - num_row) + abs(col - num_col)))


    if 'Q' in board_state:
      index = board_state.index("Q")
      row,col = divmod(index,cols)
      for char in board_state:
        if char in ['2','3']:
          num_index = board_state.index(char)
          num_row,num_col = divmod(num_index,cols)
          q_distance = min(q_distance,(abs(row - num_row) + abs(col - num_col)))

    return min(p_distance,q_distance)    

import heapq
def Astar(board_map: List[str]) -> Tuple[str, str]:
    p_steps, q_steps = "",""

    cols = len(board_map[0])-1
    start_state = ''.join(s[:-1] for s in board_map) + '%'

    frontier = [] 
    heapq.heappush(frontier,(0,0,0,start_state,'','','',''))
    visited = {}
    
    i = 1
    while frontier:
      node = heapq.heappop(frontier)
      if node[3] in visited:
        if node[0] < visited[node[3]]:
          visited[node[3]] = node[0] 
        else:
          continue

      visited[node[3]] = node[0]
      # check for goal state 
      if '1' not in node[3] and '2' not in node[3] and '3' not in node[3]:
        return node[6],node[7]
           
      # get neighbors
      for board_state,agent,move,psteps,qsteps in get_successors(node[3],cols,node[6],node[7]):
        cost = node[1]+1
        h = heuristic(board_state,cols)
        f = cost+h
        heapq.heappush(frontier, (f,cost,h,board_state,agent, move,psteps,qsteps))

      i+=1
      

### Utility Functions

In [ ]:
def read_file(fileaddr):
    with open(fileaddr) as file:
        table = file.readlines()
    return table

def print_map(board_map):
    print(f"----------- Map -----------")
    for line in board_map:
        print(line, end="")
    print("\n---------------------------")

def print_agent_steps(steps, agent_name):
    print(f"Agent {agent_name}: Num of Steps: {len(steps)} - Steps: {steps}") # steps should be a string of U, R, D, L characters for Up, Right, Down, and Left movements, respectively. An example would be 'URRURRUL'

def print_steps_time(p_steps, q_steps, elpased_time, algorithm_name):
    print(f"----------- {algorithm_name} -----------")
    print_agent_steps(p_steps, "P")
    print_agent_steps(q_steps, "Q")
    print(f"Elapsed Time: {elpased_time} Seconds")
    print("----------------------------")

### Running The Code - Test 1

In [ ]:
if __name__=="__main__":
    test_case_number = 1 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")
    print_map(board_map)

    # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

----------- Map -----------
%%%%%%%%%%%%%
%....%2%....%
%...3Q......%
%.%%%%%.%%%%%
%1......%.%.%
%%%.%%..%1%.%
%....%....%.%
%3.....P%...%
%.%.%%%..%%.%
%...%1......%
%%%%%%%%%%%%%
---------------------------
----------- IDS -----------
Agent P: Num of Steps: 29 - Steps: URRUDLLDDDLLRRUULLLLLLURRUULL
Agent Q: Num of Steps: 4 - Steps: LRRU
Elapsed Time: 139.36002683639526 Seconds
----------------------------
----------- A* -----------
Agent P: Num of Steps: 29 - Steps: DDLLRRUUURRUDLLLDLLLLLURRUULL
Agent Q: Num of Steps: 4 - Steps: LRRU
Elapsed Time: 25.64184832572937 Seconds
----------------------------


### Running The Code - Test 2

In [ ]:
if __name__=="__main__":
    test_case_number = 2 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")
    print_map(board_map)

    # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

----------- Map -----------
%%%%%%
%...1%
%....%
%...P%
%%.%%%
%...3%
%1...%
%%%.%%
%3..Q%
%%%%%%
---------------------------
----------- IDS -----------
Agent P: Num of Steps: 14 - Steps: UUDDLLDDRRDLLL
Agent Q: Num of Steps: 3 - Steps: LLL
Elapsed Time: 0.5905859470367432 Seconds
----------------------------
----------- A* -----------
Agent P: Num of Steps: 14 - Steps: UULLDDDDRRLLLD
Agent Q: Num of Steps: 3 - Steps: LLL
Elapsed Time: 0.49897027015686035 Seconds
----------------------------


### Running The Code - Test 3

In [ ]:
if __name__=="__main__":
    test_case_number = 3 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")
    print_map(board_map)

    # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

----------- Map -----------
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%...3Q.......2...%......%%%%.%...%%P1%%%.%......%
%.%%%%%.%%.%%%.....%..%.....3..........%..%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
---------------------------
----------- IDS -----------
Agent P: Num of Steps: 10 - Steps: RDLLLLLLLL
Agent Q: Num of Steps: 10 - Steps: LRRRRRRRRR
Elapsed Time: 2.461320161819458 Seconds
----------------------------
----------- A* -----------
Agent P: Num of Steps: 10 - Steps: RLDLLLLLLL
Agent Q: Num of Steps: 10 - Steps: LRRRRRRRRR
Elapsed Time: 0.43866610527038574 Seconds
----------------------------


### Running The Code - Test 4

In [ ]:
if __name__=="__main__":
    test_case_number = 4 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")
    print_map(board_map)

    # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

----------- Map -----------
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%....%........%.............%
%....%%%%.%%%%%%.%..%.%.%%..%
%....%......%..%.%..%.%..%..%
%....P...........%......%%%%%
%.%%%%%.%%%%%%..%%%%.%......%
%.......%....%........%.%%%.%
%.%.%..1........3..%.%%.%%%.%
%....%........%%%%..%....Q%.%
%.......%...%..%...2........%
%.%.%%.....%.%.......%%..%..%
%%%.%%..%.%%%%%%%.%%%%%.%...%
%...%.......................%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
---------------------------
----------- IDS -----------
Agent P: Num of Steps: 5 - Steps: RRDDD
Agent Q: Num of Steps: 12 - Steps: DLLLLLLULULL
Elapsed Time: 5.709825038909912 Seconds
----------------------------
----------- A* -----------
Agent P: Num of Steps: 5 - Steps: RRDDD
Agent Q: Num of Steps: 12 - Steps: LLLLDLLULULL
Elapsed Time: 6.623495578765869 Seconds
----------------------------


### Running The Code - Test 5

In [ ]:
if __name__=="__main__":
    test_case_number = 5 # Enter the test case number here
    board_map = read_file(fileaddr = f"Test Cases/test{test_case_number}")
    print_map(board_map)

    # Running IDS
    tick = time.time()
    p_steps, q_steps = IDS(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "IDS")

    # Running A* Search
    tick = time.time()
    p_steps, q_steps = Astar(board_map)
    tock = time.time()
    print_steps_time(p_steps, q_steps, tock-tick, "A*")

----------- Map -----------
%%%%%%
%....%
%....%
%...P%
%%.%%%
%..23%
%1.1.%
%%%.%%
%...Q%
%%%%%%.
---------------------------
----------- IDS -----------
Agent P: Num of Steps: 8 - Steps: LLDDDRLL
Agent Q: Num of Steps: 5 - Steps: LUUUR
Elapsed Time: 0.09187436103820801 Seconds
----------------------------
----------- A* -----------
Agent P: Num of Steps: 8 - Steps: LLDDDRLL
Agent Q: Num of Steps: 5 - Steps: LUUUR
Elapsed Time: 0.04330015182495117 Seconds
----------------------------
